## 通过能耗数据反算总阻力RT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from class_total_resistance_estimatoin import *

<!-- # 求P_effective
# 定义
etaR = 1.0 # 准确计算较复杂，但整体波动小，经验值是1-1.07或者0.98
etaO = 0.60 # 可以算，但需要知道V_A和thruster force T的具体意义，经验值0.55-0.7
etaS = 0.99 # 无计算公式，取经验值，0.99，0.98，0.95-0.96三种情况
etaH = 1.1 # 要计算t, w，1-t/1-w,需要知道力和速度的关系。
SOFC = 200 # g/kwh
eta = etaR*etaO*etaS*etaH

# 求P_effective
df_fuel = pd.read_csv('/raw_data/mrv.csv')
df_fuel['input_date'] = pd.to_datetime(df_fuel['input_date'],format='ISO8601')
df_fuel = df_fuel[df_fuel['daily_voyage_time_mrv'] >= 24]
df_fuel = df_fuel.dropna(axis=0, subset = ['daily_fuel_mrv'])
df_fuel = df_fuel.reset_index(drop=True)
df_fuel = df_fuel.drop(4747) # 这一行重复了
df_fuel = df_fuel.reset_index(drop=True)
years = df_fuel['input_date'].dt.year
months = df_fuel['input_date'].dt.month
days = df_fuel['input_date'].dt.day
df_fuel['daily_fuel_mrv'] = df_fuel['daily_fuel_mrv']*1000/24/SOFC*1000 # 一天的油耗(吨)乘10的3次方变为kg，转化成每个小时消耗,kg/h，再根据原公式除以(SOFC*10-3)获得功率，单位千瓦
df_fuel['daily_fuel_mrv'] = df_fuel['daily_fuel_mrv']*eta # 计算P effective 单位千瓦
df_fuel = df_fuel.rename(columns = {'daily_fuel_mrv': 'P_e'})
df_fuel.to_csv('/content/Ship-fuel-prediction/processed/mrv_P_effective.csv') -->

In [ ]:
# # 此时还需要知道速度才能求Rtotal,先和AIS数据合并
# df_ais = pd.read_csv('/processed/AIS_hour.csv')
# df_ais = df_ais.drop("Unnamed: 0", axis=1)
# df_ais['postime'] = pd.to_datetime(df_ais['postime'],format='ISO8601')

# years = df_ais['postime'].dt.year
# months = df_ais['postime'].dt.month
# days = df_ais['postime'].dt.day
# num = df_ais['num']
# for i in range(df_ais.shape[0]):
#   temp = df_fuel[(df_fuel['input_date'].dt.year == years[i]) & (df_fuel['input_date'].dt.month == months[i]) & (df_fuel['input_date'].dt.day == days[i]) & (df_fuel['num'] == num[i])]
#   pe = temp['P_e']
#   if not pe.empty:
#     df_ais.loc[i, 'Pe'] = pe.values
#   else:
#     df_ais.loc[i, 'Pe'] = np.nan

# df_ais

In [ ]:
# 计算Rtotal

df_ais = pd.read_csv('../processed/AIS_hour_with_P.csv')
df_ais = df_ais.drop("Unnamed: 0", axis=1)

def Pe2Rtotal(row):
    cu = row['stream_val']*np.sin(np.deg2rad(row['stream_direction']))
    cv = row['stream_val']*np.cos(np.deg2rad(row['stream_direction']))
    sog = row['SOG']
    Pe = row['Pe']
    heading_ship = row['heading']
    V_water = speedGPS2Water(sog, heading_ship, cu, cv)*0.5144 # knot to m/s
    r_total = Pe/V_water

    return r_total # 单位千牛
    
df_ais['R_total'] = df_ais.apply(Pe2Rtotal, axis=1)
df_ais

/tmp/ipykernel_8290/1069338342.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  r_total = Pe/V_water


,SOG,heading,draught,wind_val,wind_direction,wave_val,wave_direction,stream_val,stream_direction,postime,num,Pe,R_total
0,13.920000,32.000000,13.3,9.366000,41.000000,2.966000,37.400000,0.480000,211.200000,2021-01-04 00:00:00,1,9719.3250,1312.121536
1,13.950000,31.500000,13.3,9.710000,40.000000,2.990000,37.000000,0.510000,206.000000,2021-01-04 01:00:00,1,9719.3250,1306.885008
2,13.466667,32.333333,13.3,11.306667,36.000000,2.923333,37.666667,0.646667,191.000000,2021-01-04 02:00:00,1,9719.3250,1342.985079
3,13.300000,37.666667,13.3,11.386667,36.666667,2.996667,37.666667,0.646667,184.333333,2021-01-04 03:00:00,1,9719.3250,1365.180895
4,13.533333,35.000000,13.3,11.323333,36.666667,3.036667,37.333333,0.620000,182.333333,2021-01-04 04:00:00,1,9719.3250,1344.299734
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146404,17.684615,209.461538,13.7,2.736154,11.307692,0.505385,37.307692,0.090769,101.769231,2024-11-30 19:00:00,13,11987.1675,1315.658244
146405,17.720000,203.600000,13.7,6.772000,37.800000,1.376000,94.600000,0.446000,260.000000,2024-11-30 20:00:00,13,11987.1675,1333.654946
146406,18.233333,202.777778,13.7,7.278889,47.111111,1.438889,91.777778,0.578889,253.222222,2024-11-30 21:00:00,13,11987.1675,1304.428652
146407,17.780000,198.600000,13.7,4.704000,28.800000,0.882000,54.000000,0.342000,147.600000,2024-11-30 22:00:00,13,11987.1675,1326.701092
